# imports

In [1]:
import pickle
from utils import split_validation, fake_parser
from srgnn_pl import SRGNN_model, SRGNN_Map_Dataset, SRGNN_sampler
import torch
import os

from torch.utils.data import DataLoader
import pytorch_lightning as pl
import wandb
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

import torch.utils.data as data_utils
import numpy as np

# data loading

In [2]:
import pandas as pd
from utils import split_validation, fake_parser, calculate_embeddings
import numpy as np

In [3]:
import pickle

In [4]:
opt=fake_parser(step=3, batchSize=128, hiddenSize=256, epoch=60, validation=True, valid_portion=0.2,
                nonhybrid=True,
               dataset='yoochoose_custom',
               pretrained_embedings=True)

In [5]:
train_data = pickle.load(open('../datasets/' + opt.dataset  + '/train.txt', 'rb'))
if opt.validation:
    train_data, valid_data = split_validation(train_data, opt.valid_portion)
else:
    test_data = pickle.load(open('../datasets/' + opt.dataset + '/test.txt', 'rb'))

if opt.dataset == 'diginetica':
    n_node = 43098
elif opt.dataset == 'yoochoose1_64' or opt.dataset == 'yoochoose1_4':
    n_node = 37484
elif opt.dataset == 'yoochoose_custom':
    n_node = 28583
elif opt.dataset == 'yoochoose_custom_augmented':
    n_node = 27809
elif opt.dataset == 'yoochoose_custom_augmented_5050':
    n_node = 27807
else:
    n_node = 310


In [6]:
if opt.pretrained_embedings:
    clicks_df=pickle.load(open(f'../datasets/{opt.dataset}/yoo_df.txt', 'rb'))
    items_in_train=pickle.load(open(f'../datasets/{opt.dataset}/items_in_train.txt', 'rb'))
    item2id=pickle.load(open(f'../datasets/{opt.dataset}/item2id.txt', 'rb'))

    embeddings = calculate_embeddings(opt, clicks_df, items_in_train, item2id, n_node)
    print('embeddingas calculated')
    del clicks_df
    del items_in_train
    del item2id

embeddingas calculated


In [7]:
train_dataset=SRGNN_Map_Dataset(train_data, shuffle=True)
del train_data
val_dataset=SRGNN_Map_Dataset(valid_data)
del valid_data

train_dataloader=DataLoader(train_dataset, 
                            #batch_size=opt.batchSize, 
                            num_workers=os.cpu_count(),  
                          #  worker_init_fn=worker_init_fn, 
                            sampler=SRGNN_sampler(train_dataset, opt.batchSize, shuffle=True, drop_last=True)
                            #drop_last=
                            )
#del train_dataset
val_dataloader=DataLoader(val_dataset, 
                          #batch_size=opt.batchSize, 
                          num_workers=os.cpu_count(), 
                          sampler=SRGNN_sampler(val_dataset, opt.batchSize, shuffle=False, drop_last=False)

                        #  worker_init_fn=worker_init_fn
                         )
#del val_dataset

data masking start
data masking 1
data masking 2
data masking 3
done masking
data masking start
data masking 1
data masking 2
data masking 3
done masking


# training

In [8]:
model=SRGNN_model(opt, n_node, 
                  init_embeddings=None,
                  **(opt.__dict__))

if opt.unfreeze_epoch>0:
    model.freeze_embeddings()

In [9]:
torch.set_float32_matmul_precision('medium')

In [10]:
wandb_logger = pl.loggers.WandbLogger(project='GNN_master',entity="kpuchalskixiv",
                                      log_model="all")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kpuchalskixiv. Use `wandb login --relogin` to force relogin


In [11]:
trainer=pl.Trainer(max_epochs=60,
                   limit_train_batches=train_dataset.length//opt.batchSize,
                   limit_val_batches=val_dataset.length//opt.batchSize,
                   callbacks=[
                              EarlyStopping(monitor="val_loss", patience=6, mode="min", check_finite=True)],
                   logger=wandb_logger
                  )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [12]:
trainer.fit(model=model, 
            train_dataloaders=train_dataloader,
            val_dataloaders=val_dataloader
            )

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type         | Params
---------------------------------------
0 | model | SessionGraph | 8.4 M 
---------------------------------------
1.1 M     Trainable params
7.3 M     Non-trainable params
8.4 M     Total params
33.479    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [ ]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
